In [2]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

/home/marco/marco/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
org_root = "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_org"
forg_root = "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_forg"

class SignaturePairsDataset(Dataset):
    def __init__(self, org_dir, forg_dir, transform=None):
        # Récupérer uniquement les fichiers .png/.jpg/.jpeg
        self.org_images = [
            os.path.join(org_dir, f) for f in os.listdir(org_dir)
            if os.path.isfile(os.path.join(org_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.forg_images = [
            os.path.join(forg_dir, f) for f in os.listdir(forg_dir)
            if os.path.isfile(os.path.join(forg_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]

        self.transform = transform

        if len(self.org_images) == 0 or len(self.forg_images) == 0:
            raise RuntimeError("Dataset vide ! Vérifie les chemins et extensions d'images.")

    def __len__(self):
        # On prend le minimum pour éviter d’aller trop loin
        return min(len(self.org_images), len(self.forg_images))

    def __getitem__(self, idx):
        # Paire positive (2 signatures originales)
        if random.random() < 0.5:
            #img1 = Image.open(self.org_images[idx]).convert("L")
            #img2 = Image.open(random.choice(self.org_images)).convert("L")
            img1 = Image.open(self.org_images[idx]).convert("RGB")
            img2 = Image.open(random.choice(self.org_images)).convert("RGB")
            label = 1
        else:
            # Paire négative (original vs forgery)
            #img1 = Image.open(self.org_images[idx]).convert("L")
            #img2 = Image.open(random.choice(self.forg_images)).convert("L")
            img1 = Image.open(self.org_images[idx]).convert("RGB")
            img2 = Image.open(random.choice(self.org_images)).convert("RGB")

            label = 0

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return (img1, img2), label


### utilisation

In [16]:
from torchvision import transforms

# Définir les transformations (resize pour ResNet18 + conversion Tensor)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Datasets
train_dataset = SignaturePairsDataset(
    "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_org",
    "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_forg",
    transform=transform
)

test_dataset = SignaturePairsDataset(
    "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_org",
    "/home/marco/Projet_du_Fin_DEtude_M2/Datasets/full_forg",
    transform=transform
)

# DataLoaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
